# Datenjournalismus in Python - 
# Eine praktische Einführung in die Programmierung


### Natalie Widmann




Wintersemester 2022 / 2023


Universität Leipzig





### Ziel

Wir visualisieren die Daten im EM-DAT Datensatz und bauen ein Länderdashboard damit.


![Datenpipeline](../imgs/datapipeline.png)

In [ ]:
# Install a pip package im Jupyter Notebook
!pip3 install pandas
!pip3 install openpyxl

In [ ]:
import pandas as pd
data = pd.read_csv('../data/clean_emdat.csv', index_col=0)

In [ ]:
data

In [ ]:
data.info()

## Das Dashboard...

## Recherchefragen

- Wie viele Todesopfer gibt es in einem Land?
- Wie stark ist ein Land von Naturkatastrophen betroffen?
- Welchen Anteil haben die unterschiedlichen Naturkatastrophentypen daran?
- Wie hat sich die Anzahl der Naturkatastrophen über die Jahre hin entwickelt?

### Wie viele Todesopfer gibt es in einem Land?

#### Wie viele Todesopfer gibt es insgesamt?

In [ ]:
data["Total Deaths"].sum()

#### Wie viele Todesopfer gibt es insgesamt in Deutschland?

In [ ]:
data['Country'] == 'Germany'

In [ ]:
data[data['Country'] == 'Germany']

#### Korrektur der Ländernamen für Deutschland

In [ ]:
# Vorkommen von Deutschland
for country in data['Country'].unique():
    if 'german' in country.lower():
        print(country)

Schreibe eine Funktion die dies bereinigt.

**Eingabe**: beliebiger Ländername (bspw. Indonesia, Belgium, Germany Fed Rep, etc.)

**Ausgabe**: der Ländername oder Germany falls einer der oben gelisteten Namen eingegeben wurde

In [ ]:
def clean_country

In [ ]:
text = 'Germany Fed Rep'
clean_text = clean_country(text)
print(text)

Anwendung der Funktion auf alle Werte einer Spalte

In [ ]:
for index, row in data.iterrows():
    data.loc[index, "Country"] = clean_country(row['Country'])

In [ ]:
help(data.loc)

In [ ]:
data[data['Country'] == 'Germany']

**Alternative Lösungsmöglichkeit** (viel übersichtlicher und effizienter)

mit `apply()` kann eine Funktion auf eine komplette Spalte oder Zeile des Dataframes angewendet werden

In [ ]:
data['Country'] = data['Country'].apply(clean_country)

In [ ]:
data[data['Country'] == 'Germany']

#### Wie viele Todesopfer gab es insgesamt in Deutschland?

In [ ]:
data[data['Country'] == 'Germany']

#### Wie viele Todesopfer gab es insgesamt in Indien?

#### Allgemeine Funktion die Gesamtzahl der Todesopfer eines Landes ausgibt

## Dashboard Teil II

### Welchen Anteil haben die unterschiedlichen Naturkatastrophentypen in Deutschland?

# Visualisierung von DataFrames

[Matplotlib](https://matplotlib.org/) is a comprehensive library for creating static, animated, and interactive visualizations in Python.

Matplotlib makes easy things easy and hard things possible.

Install matplotlib:

In [ ]:
!pip3 install --upgrade pip
!pip3 install --upgrade Pillow
!pip3 install matplotlib

import matplotlib.pyplot as plt

In [ ]:
germany_data = data[data['Country'] == 'Germany']

In [ ]:
germany_data['Disaster Type'].value_counts()

In [ ]:
germany_data['Disaster Type'].value_counts().plot()

In [ ]:
germany_data['Disaster Type'].value_counts().plot(kind='pie')

## Recherchefragen

#### Welche Naturkatastrophen fordern die meisten Todesopfer?

`.groupby()` gruppiert einen DataFrame nach den Werten einer oder mehreren Spalten.

Die Spalten nach denen man Gruppieren möchte werden als Argument übergeben. Danach folgt die gewünschte Berechnung auf dieser Gruppe. Das Ergebnis wird als DataFrame zurückgegeben. 

In [ ]:
data

In [ ]:
data['Total Deaths'].sum()

In [ ]:
data.groupby('Disaster Type')['Total Deaths'].sum()

In [ ]:
data.groupby('Disaster Type').sum()

`.groupby()` kann auch auf mehrere Spalten gleichzeitig angewendet werden

In [ ]:
data.groupby(['Disaster Type', 'Disaster Subtype'])['Total Deaths'].sum()

### Visualisierung

In [ ]:
data.groupby('Disaster Type')['Total Deaths'].sum().plot(kind='pie')

#### Welche Naturkatastrophen fordern die meisten Todesopfer in Deutschland?

#### Generelle Funktion

In [ ]:
country = 'Germany'
country_data = data[data['Country'] == country]
country_data.groupby('Disaster Type')['Total Deaths'].sum().plot(kind='pie')

In [ ]:
def plot_death_by_disastertype

## Dashboard Teil III

#### Wie hat sich die Anzahl der Naturkatastrophen über die Jahre hin entwickelt?

In [ ]:
germany_data

**Berechne für jedes Jahr die Summe aus allen `Events`**

In [ ]:
yearly_events = data.groupby('Year')['Total Events'].sum()
yearly_events.plot(kind='line', x='Year', y='Total Events', title='Anzahl an Naturkatastrophen pro Jahr')

**Plotte für Deutschland die Entwicklung der Anzahl an Naturkatastrophen über die Jahre hinweg**

**Generelle Funktion**

## Länderdashboard

In [ ]:
def plot_pie(country_data):
    country_data.groupby('Disaster Type')['Total Deaths'].sum().plot(kind='pie', title='Anteil an getöten Menschen nach Naturkatastrophentyp')
    plt.show()

In [ ]:
def plot_time_evolution(country_data):
    yearly_events = country_data.groupby('Year')['Total Events'].sum()
    yearly_events.plot(kind='line', x='Year', y='Total Events', title='Anzahl an Naturkatastrophen pro Jahr')
    plt.show()

In [ ]:
def death_overview(country_data):
    total_deaths = country_data["Total Deaths"].sum()
    print(f'Getötete Menschen seit 1900: {total_deaths:,.0f}')

In [ ]:
def compute_anteil(country_total, world_total):
    anteil = round(country_total / (world_total / 100.0), 2)
    print(f'{anteil}% aller Menschen die weltweit von Naturkatastrophen betroffen sind leben hier.')

In [ ]:
def analyze(data, country):
    print(f'Naturkatastrophen in {country.upper()} \n')
    country_data = data[data['Country'] == country]
    
    compute_anteil(country_data['Total Affected'].sum(), data['Total Affected'].sum())
    death_overview(country_data)
    plot_pie(country_data)
    plot_time_evolution(country_data)

In [ ]:
analyze(data, 'Bangladesh')

### Überblick über die Welt

Wie können wir die selbstdefinierte Funktion `analyze()` anpassen, so dass auch eine Gesamtstatistik, die alle Länder der Welt enthält abgefragt werden kann?

In [ ]:
def analyze(data, country):
    print(f'Naturkatastrophen in {country.upper()} \n')
    country_data = data[data['Country'] == country]
    
    compute_anteil(country_data['Total Affected'].sum(), data['Total Affected'].sum())
    death_overview(country_data)
    plot_pie(country_data)
    plot_time_evolution(country_data)

In [ ]:
analyze(data, 'world')

# Zeit für Feedback



Link: https://ahaslides.com/HP3D5

![Feedback QR Code](../imgs/qrcode_vl7.png)

